In [4]:
import os
import pandas as pd
import numpy as np
import  datetime 

directory = r'/Users/meiyewang/Desktop/code/data/windsolar/ACTUAL/2019'
df1=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df2=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df3=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df4=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df5=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
for filename in os.listdir(directory):
    if filename.endswith(".csv") :
        df=pd.read_csv(os.path.join(directory, filename))
        df1=pd.concat([df1,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df2=pd.concat([df2,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df3=pd.concat([df3,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df4=pd.concat([df4,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df5=pd.concat([df5,df[(df['TRADING_HUB']=='ZP26') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
    else:
        continue

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(8749, 3)
(8749, 3)
(8749, 3)
(8749, 3)
(8749, 3)


In [1]:
import os, csv, operator, copy, time, random
import numpy as np
import datetime as dt
import pandas as pd

def importFuelPrices(fuelPriceScenario):
    fuelPrices = pd.read_csv(os.path.join('Data','Energy_Prices_Electric_Power.csv'),skiprows=4,index_col=0)
    fuelPrices = fuelPrices[[col for col in fuelPrices if fuelPriceScenario in col]]
    fuelPrices.columns = [col.split(':')[0] for col in fuelPrices.columns]
    return fuelPrices


    # add drop storage 
    # nameplate capacity is only for storage

def importEIA860():
    dir860 = os.path.join('Data','EIA860')
    gens860 = pd.read_excel(os.path.join(dir860,'3_1_Generator_Y2019.xlsx'),sheet_name='Operable',header=1)
    sto860 = pd.read_excel(os.path.join(dir860,'3_4_Energy_Storage_Y2019.xlsx'),sheet_name='Operable',header=1)
    plants860 = pd.read_excel(os.path.join(dir860,'2___Plant_Y2019.xlsx'),sheet_name='Plant',header=1)
    # genFleet = gens860.merge(plants860[['Plant Code','Latitude','Longitude','Balancing Authority Code']],on='Plant Code',how='left')
    # genFleet = genFleet.merge(sto860[['Plant Code','Generator ID','Nameplate Energy Capacity (MWh)','Maximum Charge Rate (MW)','Maximum Discharge Rate (MW)']],on=['Plant Code','Generator ID'],how='left')
    return gens860,plants860,sto860

################################################################################
#COMPRESS FLEET BY COMBINING SMALL UNITS
def performFleetCompression(genFleet):
    maxSizeToCombine,maxCombinedSize,firstYr,lastYr,stepYr = 75,300,1975,2026,10
    rowsToDrop,rowsToAdd = list(),list()
    for fuel in ['Landfill Gas','Distillate Fuel Oil','MSW','Natural Gas','Biomass','Non-Fossil Waste','Fossil Waste','Residual Fuel Oil']:
        fuelRows = genFleet.loc[(genFleet['FuelType']==fuel) & (genFleet['Capacity (MW)']<maxSizeToCombine) & (genFleet['PlantType']!='Combined Cycle')]
        yearIntervals = [yr for yr in range(firstYr,lastYr,stepYr)]
        for endingYear in yearIntervals:
            beginningYear = 0 if endingYear == firstYr else endingYear-stepYr
            fuelRowsYears = fuelRows.loc[(fuelRows['On Line Year']>beginningYear) & (fuelRows['On Line Year']<=endingYear)]
            if fuelRowsYears.shape[0]>1: 
                runningCombinedSize,rowsToCombine = 0,list()
                for index, row in fuelRowsYears.iterrows():
                    if (runningCombinedSize + row['Capacity (MW)'] > maxCombinedSize):
                        newRow,idxsToDrop = addCombinedRowsToFleet(genFleet,rowsToCombine)
                        rowsToAdd.append(newRow),rowsToDrop.extend(idxsToDrop)
                        runningCombinedSize,rowsToCombine = row['Capacity (MW)'],[row]
                    else:
                        runningCombinedSize += row['Capacity (MW)']
                        rowsToCombine.append(row)
                if len(rowsToCombine)>1: 
                    newRow,idxsToDrop = addCombinedRowsToFleet(genFleet,rowsToCombine)
                    rowsToAdd.append(newRow),rowsToDrop.extend(idxsToDrop)    
    genFleet.drop(index=rowsToDrop,inplace=True)
    genFleet = genFleet.append(pd.DataFrame(rowsToAdd))
    genFleet.reset_index(drop=True,inplace=True)
    return genFleet
                
def addCombinedRowsToFleet(genFleet,rowsToCombine):
    rowsToCombine = pd.DataFrame(rowsToCombine)
    capacWts = rowsToCombine['Capacity (MW)']/rowsToCombine['Capacity (MW)'].sum()
    newRow = rowsToCombine.iloc[0].copy()
    newRow['Capacity (MW)'] = rowsToCombine['Capacity (MW)'].sum()
    newRow['On Line Year'] = rowsToCombine['On Line Year'].median()
    for p in ['CO2EmRate(lb/MMBtu)','Heat Rate (Btu/kWh)']: #'NOxEmRate(lb/MMBtu)','SO2EmRate(lb/MMBtu)'
        newRow[p] = (rowsToCombine[p]*capacWts).sum()
    newRow['ORIS Plant Code'],newRow['Unit ID'] = genFleet['ORIS Plant Code'].max()+1,newRow['Unit ID']+'COMBINED'
    return newRow,rowsToCombine.index
################################################################################

################ CONVERT DOLLAR YEARS
#Convert dollar years to 2012 dollars
#CPI from Minneapolis Fed, https://www.minneapolisfed.org/community/teaching-aids/
#cpi-calculator-information/consumer-price-index-and-inflation-rates-1913.
#Inputs: name of parameter of dollar year to convert, parameter value (cost)
#Outputs: cost in 2012 dollars
def convertCostToTgtYr(paramName,cost):    
    paramDollarYears = {'startup':2011,'vom':2012,'fom':2012,'occ':2012,'fuel':2019,'tgt':2012}
    targetDollarYear = 2012
    cpiValues = {2015:237,2014:236.7,2013:233,2012:229.6,2011:224.9,2010:218.1,
            2009:214.5,2008:215.3,2007:207.3,2006:201.6,2005:195.3,2019:255.657}
    return doConversion(paramName,cost,paramDollarYears,targetDollarYear,cpiValues)

#Convert dollar year
def doConversion(paramName,cost,paramDollarYears,targetDollarYear,cpiValues):
    paramDollarYear = paramDollarYears[paramName]
    (cpiTgtYear,cpiParamYear) = (cpiValues[targetDollarYear],cpiValues[paramDollarYear])
    return cost*cpiTgtYear/cpiParamYear
################

################ READ AND WRITE CSV LISTS
#Read CSV to 2d list
#Input: full file name including dir (str)
#Output: 2d list
def readCSVto2dList(fileNameWithDir):
    with open(fileNameWithDir,'r') as f:
        f = csv.reader(f)
        f = list(f)
    return f

#Write 2d list to CSV
#Input: 2d list, full file name including dir & .csv (str)
def write2dListToCSV(list2d,fileNameWithDir):
    fullFileName=fileNameWithDir
    with open(fullFileName, 'w', newline='') as csvfile:
        w = csv.writer(csvfile)
        w.writerows(list2d)
################ 

################################################################################
#ADD VARIABLE AND FIXED O&M COSTS
#Based on plant type
def addVOMandFOM(genFleet):
    vomData = pd.read_csv(os.path.join('Data','VOMValues.csv'),index_col=0)
    genFleet = genFleet.merge(vomData[['FOM(2012$/MW/yr)','VOM(2012$/MWh)']],left_on='PlantType',right_index=True,how='left')
    genFleet['VOM(2012$/MWh)'] = convertCostToTgtYr('vom',genFleet['VOM(2012$/MWh)'])
    genFleet['FOM(2012$/MW/yr)'] = convertCostToTgtYr('fom',genFleet['FOM(2012$/MW/yr)'])
    return genFleet
################################################################################

################################################################################
#ADD UNIT COMMITMENT PARAMETERS
#Based on fuel and plant type; data from PHORUM
def addUnitCommitmentParameters(genFleet,fname):
    ucData = readCSVto2dList(os.path.join('Data',fname))
    for ucHeader in ['MinDownTime(hrs)','RampRate(MW/hr)','StartCost($)','MinLoad(MWh)']:# min stable load
        genFleet[ucHeader] = genFleet['Capacity (MW)'] if (ucHeader in ['RampRate(MW/hr)','MinLoad(MWh)']) else 0
        phorumParamName = mapHeadersToPhorumParamNames()[ucHeader]
        for index,row in genFleet.iterrows():
            (fuel,plantType,size) = (row['FuelType'],row['PlantType'],float(row['Capacity (MW)']))
            phorumValue = getMatchingPhorumValue(ucData,fuel,plantType,size,phorumParamName)
            if phorumValue is not None: #input files don't have all plant types
                valToAdd = phorumValue if ucHeader == 'MinDownTime(hrs)' else phorumValue*size
                if ucHeader == 'StartCost($)': valToAdd = convertCostToTgtYr('startup',valToAdd)
                genFleet.loc[index,ucHeader]=valToAdd
    return genFleet

def getMatchingPhorumValue(ucData,fuel,plantType,size,paramName):
    if plantType == 'Fuel Cell': plantType = 'Combustion Turbine'
    fuel = mapFuels()[fuel]
    phorumPropertyNameCol = ucData[0].index('PropertyName')
    phorumFuelCol = ucData[0].index('Fuel')
    phorumPlantTypeCol = ucData[0].index('PlantType')
    phorumLowerSizeCol = ucData[0].index('LowerPlantSizeLimit')
    phorumUpperSizeCol = ucData[0].index('UpperPlantSizeLimit')
    phorumValueCol = ucData[0].index('PropertyValue')
    phorumProperties = [row[phorumPropertyNameCol] for row in ucData[1:]]
    phorumFuels = [row[phorumFuelCol] for row in ucData[1:]]
    phorumPlantTypes = [row[phorumPlantTypeCol] for row in ucData[1:]]
    phorumLowerSizes = [int(row[phorumLowerSizeCol]) for row in ucData[1:]]
    phorumUpperSizes = [int(row[phorumUpperSizeCol]) for row in ucData[1:]]
    phorumValues = [float(row[phorumValueCol]) for row in ucData[1:]]
    for idx in range(len(phorumProperties)):
        if (phorumProperties[idx] == paramName and phorumFuels[idx] == fuel and 
            (phorumPlantTypes[idx] in plantType or phorumPlantTypes[idx] == 'All') and 
            (phorumLowerSizes[idx] <= size and phorumUpperSizes[idx] > size)):
            return float(phorumValues[idx])

#Return dictionary of fleet fuel : UC fuel
def mapFuels():
    return {'Bituminous': 'Coal', 'Petroleum Coke': 'Pet. Coke',
        'Subbituminous': 'Coal', 'Lignite': 'Coal', 'Natural Gas': 'NaturalGas',
        'Distillate Fuel Oil': 'Oil', 'Hydro': 'Hydro', 'Landfill Gas': 'LF Gas',
        'Biomass': 'Biomass', 'Solar': 'Solar', 'Non-Fossil Waste': 'Non-Fossil',
        'MSW': 'MSW', 'Pumped Storage': 'Hydro', 'Residual Fuel Oil': 'Oil',
        'Wind': 'Wind', 'Nuclear Fuel': 'Nuclear', 'Coal': 'Coal','Energy Storage':'Storage',
        'Hydrogen':'Hydrogen','Storage':'Storage','Geothermal':'Geothermal','Fossil Waste':'Fwaste'}
    #EIA860 fuels
    # fleetFuelToPhorumFuelMap = {'BIT':'Coal','PC':'Pet. Coke','SGC':'Coal',
    #         'SUB':'Coal','LIG':'Coal','RC':'Coal','NG':'NaturalGas','OG':'NaturalGas',
    #         'BFG':'NaturalGas','DFO':'Oil','RFO':'Oil','KER':'Oil','BLQ':'Oil',
    #         'WAT':'Hydro','LFG':'LF Gas','OBG':'LF Gas','WH':'NaturalGas','PUR':'NaturalGas',
    #         'AB':'Biomass','WDS':'Biomass','SUN':'Solar','Non-Fossil Waste':'Non-Fossil',
    #         'MSW':'MSW','WND':'Wind','NUC':'Nuclear','MWH':'MWH'}

def mapHeadersToPhorumParamNames():
    return {'MinDownTime(hrs)':'Min Down Time','RampRate(MW/hr)':'Ramp Rate',
            'StartCost($)':'Start Cost','MinLoad(MWh)':'Min Stable Level'}
################################################################################

################################################################################
#ADD FUEL PRICES
def addFuelPrices(genFleet,currYear,fuelPrices):
    fuelPrices = fuelPrices.loc[currYear] if currYear in fuelPrices.index else fuelPrices.iloc[-1]
    fuelPrices = convertCostToTgtYr('fuel',fuelPrices)
    prices = fuelPrices.to_dict()
    fuelMap = mapFuelsToAEOPrices()
    genFleet['FuelPrice($/MMBtu)'] = [prices[fuelMap[f]] if (f in fuelMap and fuelMap[f] in prices) else (prices[f] if f in prices else 0) for f in genFleet['FuelType']]
    return genFleet


def mapFuelsToAEOPrices():
    return {'Bituminous': 'Steam Coal', 'Petroleum Coke': 'Steam Coal','Coal':'Steam Coal',
        'Subbituminous': 'Steam Coal', 'Lignite': 'Steam Coal','Nuclear Fuel': 'Uranium'}
################################################################################

################################################################################
#ADD RANDOM OP COST ADDER TO FLEET IN NEW COLUMN
#Add to all fuel types. Use value of 0.05 - makes up ~0.03% on average of fleet. 
#Max addition to op cost of gen in fleet is 0.19%.  
def addRandomOpCostAdder(genFleet,ocAdderMin=0,ocAdderMax=0.05):
    random.seed()
    genFleet['RandOpCostAdder($/MWh)'] = pd.Series(np.random.uniform(ocAdderMin,ocAdderMax,genFleet.shape[0]))
    return genFleet
################################################################################

################################################################################
def calcOpCost(genFleet):
    genFleet['OpCost($/MWh)'] = genFleet['FuelPrice($/MMBtu)']*genFleet['Heat Rate (Btu/kWh)']/1000+genFleet['VOM(2012$/MWh)']+genFleet['RandOpCostAdder($/MWh)']
    return genFleet
################################################################################

################################################################################
#ADD REG OFFER COST AND ELIGIBILITY
def addRegResCostAndElig(genFleet,regElig,regCostFrac):
    genFleet['RegOfferElig'] = 0
    genFleet.loc[genFleet['PlantType'].str.contains('|'.join(regElig)),'RegOfferElig'] = 1
    genFleet['RegOfferCost($/MW)'] = regCostFrac*genFleet['OpCost($/MWh)']*genFleet['RegOfferElig']
    return genFleet
################################################################################

################################################################################
def addEmissionsRates(genFleet):
    emissionRates = pd.read_excel(os.path.join('Data','co2_vol_mass_updated.xls'),sheet_name='Sheet1',index_col=0,skiprows=2,usecols='A,F')
    emissionRates = emissionRates[emissionRates.columns[0]] #convert to Series
    fuelMap = fuelMapEmissions()
    genFleet['CO2EmRate(lb/MMBtu)'] = [emissionRates[fuelMap[f]] if (f in fuelMap and fuelMap[f] in emissionRates) else (emissionRates[f] if f in emissionRates else 0) for f in genFleet['FuelType']]
    return genFleet

def fuelMapEmissions():
    return {'MSW':'Municiple Solid Waste','Biomass':'Municiple Solid Waste','Landfill Gas':'Natural Gas',
            'Distillate Fuel Oil':'Other petroleum & miscellaneous','Residual Fuel Oil':'Other petroleum & miscellaneous',
            'Waste Coal':'Bituminous','Fossil Waste':'Other petroleum & miscellaneous','Non-Fossil Waste':'Other petroleum & miscellaneous',
            'Petroleum Coke':'Petroleum coke'}
################################################################################

######################## OLD CODE ##############################################
################################################################################
#ADD EMISSION RATES FROM EGRID TO GENERATOR FLEET
#Adds eGRID emissions rates to generator fleet
#IN: generator fleet (2d list), states for analysis (1d list)
def addEmissionsRatesOLD(baseGenFleet,statesForAnalysis,runLoc):
    (egridBoiler,egridPlant) = importeGridData(statesForAnalysis,runLoc)
    emsHeadersToAdd=["NOxEmRate(lb/MMBtu)","SO2EmRate(lb/MMBtu)",
                  "CO2EmRate(lb/MMBtu)"]
    addHeaders(baseGenFleet,emsHeadersToAdd)    
    addEmissionsRatesValues(baseGenFleet,egridBoiler,egridPlant)
    fillMissingEmissionsRates(baseGenFleet,emsHeadersToAdd) 

#Fills missing generator em rates w/ average for gens w/ same fuel and plant type.
#IN: generator fleet (2d list), emissions headers to add (1d list)
def fillMissingEmissionsRates(baseGenFleet,emsHeadersToAdd):
    #Get headers and columns
    headersToColsMapBase = mapHeadersToCols(baseGenFleet)
    plantTypeCol = headersToColsMapBase['PlantType']
    fuelTypeCol = headersToColsMapBase['FuelType']
    noxCol = headersToColsMapBase[emsHeadersToAdd[0]]
    so2Col = headersToColsMapBase[emsHeadersToAdd[1]]
    co2Col = headersToColsMapBase[emsHeadersToAdd[2]]
    #Find and fill missing emissions rates values
    for idx in range(1,len(baseGenFleet)):
        if baseGenFleet[idx][noxCol]=='NA':
            (plantType,fuelType) = (baseGenFleet[idx][plantTypeCol],
                                    baseGenFleet[idx][fuelTypeCol])
            [nox,so2,co2] = getEmsRatesOfMatchingFuelAndPlantType(baseGenFleet,plantType,
                                                                  fuelType,emsHeadersToAdd)
            [avgnox,avgso2,avgco2] = [avgListVals(nox),avgListVals(so2),avgListVals(co2)]
            baseGenFleet[idx][noxCol]=avgnox
            baseGenFleet[idx][so2Col]=avgso2
            baseGenFleet[idx][co2Col]=avgco2   

#Gets emissions rates of generators w/ given plant & fuel type
#IN: generator fleet (2d list), plant and fuel type (str), em rate headers (1d list)
#OUT: NOx, SO2 and CO2 emissions rates (1d lists)
def getEmsRatesOfMatchingFuelAndPlantType(baseGenFleet,plantType,fuelType,emsHeadersToAdd):
    #Get headers
    headersToColsMapBase = mapHeadersToCols(baseGenFleet)
    noxCol = headersToColsMapBase[emsHeadersToAdd[0]]
    so2Col = headersToColsMapBase[emsHeadersToAdd[1]]
    co2Col = headersToColsMapBase[emsHeadersToAdd[2]]
    #Get cols w/ matching fuel & plant type
    matchingRowIdxs = getMatchingRowsFuelAndPlantType(baseGenFleet,plantType,fuelType,
                                                      noxCol)
    #If can't find on fuel & plant type, try just fuel type
    if matchingRowIdxs==[]:
        matchingRowIdxs = getMatchingRowsFuelType(baseGenFleet,fuelType,noxCol)
    #If still can't get emissions rate, then ues other plant & fuel type:
    #LFG - NGCT, MSW - biomass, gas & oil O/G Steam - gas O/G Steam, Non-fossil waste - 
    if matchingRowIdxs==[] and fuelType=='Landfill Gas':
        matchingRowIdxs = getMatchingRowsFuelAndPlantType(baseGenFleet,'Combustion Turbine',
                                                          'Natural Gas',noxCol)
    elif matchingRowIdxs==[] and fuelType=='MSW':
        matchingRowIdxs = getMatchingRowsFuelAndPlantType(baseGenFleet,'Biomass',
                                                          'Biomass',noxCol)
    elif matchingRowIdxs==[] and fuelType=='Natural Gas& Distillate Fuel Oil& Residual Fuel Oil':
        matchingRowIdxs = getMatchingRowsFuelAndPlantType(baseGenFleet,'O/G Steam',
                                                          'Natural Gas',noxCol)
    elif matchingRowIdxs==[] and fuelType=='Non-Fossil Waste':
        matchingRowIdxs = getMatchingRowsFuelAndPlantType(baseGenFleet,'Biomass',
                                                          'Biomass',noxCol)
    #Get emissions rates of matching rows
    [nox,so2,co2] = [[],[],[]]
    for rowIdx in matchingRowIdxs:
        row = baseGenFleet[rowIdx]
        nox.append(row[noxCol])
        so2.append(row[so2Col])
        co2.append(row[co2Col])
    return [nox,so2,co2]

#Gets row indexes in generator fleet of generators that match given plant & fuel type,
#filtering out units w/ no emissions rate data.
#IN: generator fleet (2d list), plant and fuel type (str), col w/ nox ems rate (int)
#OUT: row indices of matching plant & fuel type (1d list)
def getMatchingRowsFuelAndPlantType(baseGenFleet,plantType,fuelType,noxCol):
    headersToColsMapBase = mapHeadersToCols(baseGenFleet)
    plantTypeCol = headersToColsMapBase['PlantType']
    fuelTypeCol = headersToColsMapBase['FuelType']
    matchingRowIdxs = []
    for idx in range(len(baseGenFleet)):
        row = baseGenFleet[idx]
        if row[plantTypeCol]==plantType and row[fuelTypeCol]==fuelType:
            if row[noxCol] != 'NA': #make sure has data!
                matchingRowIdxs.append(idx)
    return matchingRowIdxs

#Gets row indexes in generator fleet of gens w/ same fuel type, filtering
#out units w/ no emissions rate data.
#IN: generator fleet (2d list), fuel type (str), col w/ nox ems rate (int)
#OUT: row indices of matching fuel type (1d list)
def getMatchingRowsFuelType(baseGenFleet,fuelType,noxCol):
    headersToColsMapBase = mapHeadersToCols(baseGenFleet)
    fuelTypeCol = headersToColsMapBase['FuelType']
    matchingRowIdxs = []
    for idx in range(len(baseGenFleet)):
        row = baseGenFleet[idx]
        if row[fuelTypeCol]==fuelType:
            if row[noxCol] != 'NA': #make sure has data!
                matchingRowIdxs.append(idx)
    return matchingRowIdxs

#Add eGRID emissions rates values to fleet, either using boiler specific 
#data for coal & o/g steam units or plant level average data. Adds
#ems rate in order of nox, so2, and co2, as set by ems headers in addEmissionsRates.
#IN: generator fleet (2d list), eGRID boiler and plant data (2d lists)
def addEmissionsRatesValues(baseFleet,egridBoiler,egridPlant):
    headersToColsMapBase = mapHeadersToCols(baseFleet)
    headersToColsMapEgridBlr = mapHeadersToCols(egridBoiler)
    headersToColsMapEgridPlnt = mapHeadersToCols(egridPlant)
    basePlantTypeCol = headersToColsMapBase['PlantType']
    noEmissionPlantTypes = ['hydro','solar pv','wind','geothermal',
                            'solar thermal','pumped storage','nuclear']
    for idx in range(1,len(baseFleet)):
        plantType = baseFleet[idx][basePlantTypeCol].lower()
        if plantType == 'coal steam':  
            [nox,so2,co2] = getBlrEmRates(baseFleet,idx,egridBoiler)
        elif plantType == 'o/g steam':
            [nox,so2,co2] = getBlrEmRates(baseFleet,idx,egridBoiler)
            if nox == 'NA': #just test on nox, but all would be na
                [nox,so2,co2] = getPlantEmRates(baseFleet,idx,egridPlant)
        elif plantType in noEmissionPlantTypes:
            [nox,so2,co2] = [0,0,0]
        else:
            [nox,so2,co2] = getPlantEmRates(baseFleet,idx,egridPlant)
        #Some plants have no emissions info, so end up w/ zero emission values - 
        #fill in 'NA' if so.
        if [nox,so2,co2] == [0,0,0] and plantType not in noEmissionPlantTypes:
            [nox,so2,co2]=['NA','NA','NA']
        baseFleet[idx].extend([nox,so2,co2])

#Look for boiler-level match of given gen in gen fleet to eGRID data, and return emissions 
#rates if find match.
#IN: gen fleet (2d list), idx for row in gen fleet (int), boiler data (2d list)
#OUT: boiler-level nox, so2 & co2 ems rates (1d list)
def getBlrEmRates(baseFleet,idx,egridBoiler):
    #Setup necessary data
    headersToColsMapBase = mapHeadersToCols(baseFleet)
    headersToColsMapEgridBlr = mapHeadersToCols(egridBoiler)
    (baseOrisCol,baseUnitCol) = (headersToColsMapBase["ORIS Plant Code"],
                                 headersToColsMapBase["Unit ID"])
    (egridOrisCol,egridBlrCol) = (headersToColsMapEgridBlr["DOE/EIA ORIS plant or facility code"],
                                  headersToColsMapEgridBlr["Boiler ID"])
    (egridBlrORISIDs,egridBlrIDs) = (colTo1dList(egridBoiler,egridOrisCol),
                                   colTo1dList(egridBoiler,egridBlrCol))    
    #eGrid ORIS IDs are given w/ .0 @ end (e.g., 5834.0). So convert to int and back to str.
    removeTrailingDecimalFromEgridORIS(egridBlrORISIDs)
    #Do mapping
    (baseOrisID,baseUnitID) = (baseFleet[idx][baseOrisCol],baseFleet[idx][baseUnitCol])
    try:
        egridBlrRow = search2Lists(egridBlrORISIDs, egridBlrIDs, baseOrisID, baseUnitID)
        [nox,so2,co2] = calculateEmissionsRatesBlr(egridBoiler,egridBlrRow)
    except:
        # print('No matching boiler for: ORIS' + str(baseOrisID) + ' Blr' + str(baseUnitID))
        [nox,so2,co2] = ['NA','NA','NA']
    return [nox,so2,co2]

#Looks for plant-level match of given unit in gen fleet to eGRID plant data,
#and returns plant-level ems rate of matching plant if found.
#IN: gen fleet (2d list), idx for row in gen fleet (int), plant data (2d list)
#OUT: plant-level nox, so2 & co2 ems rate (1d list)
def getPlantEmRates(baseFleet,idx,egridPlant):
    #Setup necessary data
    headersToColsMapBase = mapHeadersToCols(baseFleet)
    headersToColsMapEgridPlnt = mapHeadersToCols(egridPlant)
    baseOrisCol = headersToColsMapBase["ORIS Plant Code"]
    egridOrisCol = headersToColsMapEgridPlnt["DOE/EIA ORIS plant or facility code"]
    egridORISIDs = colTo1dList(egridPlant,egridOrisCol)   
    #eGrid ORIS IDs are given w/ .0 @ end (e.g., 5834.0). So convert to int and back to str.
    # removeTrailingDecimalFromEgridORIS(egridORISIDs)
    #Do mapping
    baseOrisID = baseFleet[idx][baseOrisCol]
    try:
        egridPlantRow = egridORISIDs.index(baseOrisID)
        [nox,so2,co2] = calculateEmissionsRatesPlnt(egridPlant,egridPlantRow)
    except:
        # print('No matching plant for: ORIS' + str(baseOrisID))
        [nox,so2,co2] = ['NA','NA','NA']
    return [nox,so2,co2]
    
#Gets boiler-level emissions rates.
#IN: eGRID boiler data (2d list), row in boiler data (int)
#OUT: boiler-level emissions rates [lb/mmbtu] (1d list)
def calculateEmissionsRatesBlr(egridBoiler,egridBoilerRow):
    scaleTonsToLbs = 2000
    #Define headers
    htInputHeader = 'Boiler unadjusted annual best heat input (MMBtu)'
    noxHeader = 'Boiler unadjusted annual best NOx emissions (tons)'
    so2Header = 'Boiler unadjusted annual best SO2 emissions (tons)'
    co2Header = 'Boiler unadjusted annual best CO2 emissions (tons)'
    #Calculate values
    headersToColsMap = mapHeadersToCols(egridBoiler)
    (htinputCol,noxCol,so2Col,co2Col) = (headersToColsMap[htInputHeader],
                                        headersToColsMap[noxHeader],
                                        headersToColsMap[so2Header],
                                        headersToColsMap[co2Header])
    blrData = egridBoiler[egridBoilerRow]
    (htInput,noxEms,so2Ems,co2Ems) = (blrData[htinputCol],blrData[noxCol],
                                      blrData[so2Col],blrData[co2Col])
    #Str nums have commas in them - use helper function to turn into numbers 
    (htInput,noxEms,so2Ems,co2Ems) = (toNum(htInput),toNum(noxEms),toNum(so2Ems),
                                      toNum(co2Ems))
    (noxEmsRate,so2EmsRate,co2EmsRate) = (noxEms/htInput*scaleTonsToLbs, 
                                          so2Ems/htInput*scaleTonsToLbs,
                                          co2Ems/htInput*scaleTonsToLbs)
    return [noxEmsRate,so2EmsRate,co2EmsRate]

#Gets plant-level ems rates.
#IN: eGRID plant data (2d list), row in plant data (int)
#OUT: plant-level nox, so2 and co2 ems rates [lb/mmbtu] (1d list)
def calculateEmissionsRatesPlnt(egridPlant,egridPlantRow):
    #Define headers
    noxEmsRateHeader = 'Plant annual NOx input emission rate (lb/MMBtu)'
    so2EmsRateHeader = 'Plant annual SO2 input emission rate (lb/MMBtu)'
    co2EmsRateHeader = 'Plant annual CO2 input emission rate (lb/MMBtu)'
    #Get values
    headersToColsMap = mapHeadersToCols(egridPlant)
    (noxCol,so2Col,co2Col) = (headersToColsMap[noxEmsRateHeader],
                              headersToColsMap[so2EmsRateHeader],
                              headersToColsMap[co2EmsRateHeader])
    plantData = egridPlant[egridPlantRow]
    (noxEmsRate,so2EmsRate,co2EmsRate) = [plantData[noxCol],plantData[so2Col],plantData[co2Col]]
    #Ems rate nums have commas - use helper func to turn into numbers
    (noxEmsRate,so2EmsRate,co2EmsRate) = (toNum(noxEmsRate),
                                          toNum(so2EmsRate),
                                          toNum(co2EmsRate))
    return [noxEmsRate,so2EmsRate,co2EmsRate]
################################################################################



################################################################################
#IMPORT DATA
#Import base generator fleet from NEEDS
#OUT: gen fleet (2d list)
def importNEEDSFleet(runLoc):
    if runLoc == 'pc': dirName = 'C:\\Users\\mtcraig\\Desktop\\EPP Research\\Databases\\NEEDS'  
    else: dirName = 'Data' 
    fileName = 'needs_v515_nocommas.csv'
    fullFileName = os.path.join(dirName,fileName)
    return readCSVto2dList(fullFileName)

def importTestFleet(runLoc):
    if runLoc == 'pc': dirName = 'C:\\Users\\mtcraig\\Desktop\\EPP Research\\Databases\\CETestFleet'
    else: dirName = ''
    fileName = 'testFleetTiny.csv'
    fullFileName = os.path.join(dirName,fileName)
    return readCSVto2dList(fullFileName)    

#Import eGRID boiler and plant level data, then isolate plants and boilers in state
#IN: states for analysis (1d list)
#OUT: eGRID boiler and plant data (2d lists)
def importeGridData(statesForAnalysis,runLoc):
    if runLoc == 'pc': dirName = 'C:\\Users\\mtcraig\\Desktop\\EPP Research\\Databases\\eGRID2015'
    else: dirName = os.path.join('Data','eGRID2015')
    egridBoiler = importeGridBoilerData(dirName)
    egridPlant = importeGridPlantData(dirName)
    egridStateColName = 'Plant state abbreviation'
    statesForAnalysisAbbrev = getStateAbbrevs(statesForAnalysis)
    isolateGensInStates(egridBoiler,statesForAnalysisAbbrev,egridStateColName)
    isolateGensInStates(egridPlant,statesForAnalysisAbbrev,egridStateColName)
    return (egridBoiler,egridPlant)

#Import eGRID boiler data and remove extra headers
#IN: directory w/ egrid data (str)
#OUT: boiler data (2d list)
def importeGridBoilerData(dirName):
    fileName = 'egrid2012_data_boiler.csv'
    fullFileName = os.path.join(dirName,fileName)
    boilerData = readCSVto2dList(fullFileName)
    boilerDataSlim = elimExtraneousHeaderInfo(boilerData,'eGRID2012 file boiler sequence number')
    return boilerDataSlim

#Import eGRID plant data and remove extra headers
#IN: directory w/ egrid data (str)
#OUT: plant data (2d list)
def importeGridPlantData(dirName):
    fileName = 'egrid2012_data_plant.csv'
    fullFileName = os.path.join(dirName,fileName)
    plantData = readCSVto2dList(fullFileName)
    plantDataSlim = elimExtraneousHeaderInfo(plantData,'eGRID2012 file plant sequence number')
    return plantDataSlim

#Eliminates first several rows in egrid CSV that has no useful info
#IN: eGRID fleet (2d list), value in col 0 in first row w/ valid data that want
#to save (str)
#OUT: eGRID fleet (2d list)
def elimExtraneousHeaderInfo(egridFleet,valueInFirstValidRow):
    for idx in range(len(egridFleet)):
        if egridFleet[idx][0]==valueInFirstValidRow:
            egridFleetSlim = copy.deepcopy(egridFleet[idx:])
    return egridFleetSlim

#Removes retired units from fleet based on input year
#IN: gen fleet (2d list), year below which retired units should be removed
#from fleet (int)
def removeRetiredUnits(baseGenFleet,retirementYearScreen):
    colName = "Retirement Year"
    colNum = baseGenFleet[0].index(colName)
    rowsToRemove= []
    for rowIdx in range(1,len(baseGenFleet)):
        retireYear = baseGenFleet[rowIdx][colNum]
        if int(retireYear)<retirementYearScreen: rowsToRemove.append(rowIdx)
    if rowsToRemove != []: removeRows(baseGenFleet,rowsToRemove)

#Isolates fleet to generators in states of interest
#IN: gen fleet (2d list), states for analyiss (1d list), col name w/ state data
#(str)
#OUT: gen fleet (2d list)
def isolateGensInStates(baseGenFleet,statesForAnalysis,colName):
    rowsToRemove = identifyRowsToRemove(baseGenFleet,statesForAnalysis,
                                        colName)
    removeRows(baseGenFleet,rowsToRemove)
    return baseGenFleet

#Isolates fleet to generators in power system of interest
#IN: gen fleet (2d list), power sys for analyiss (1d list)
#OUT: gen fleet (2d list)
def isolateGensInPowerSystem(baseGenFleet,powerSystemsForAnalysis):
    colName = "Region Name"
    rowsToRemove = identifyRowsToRemove(baseGenFleet,powerSystemsForAnalysis,
                                        colName)
    removeRows(baseGenFleet,rowsToRemove)
    return baseGenFleet
################################################################################


################################################################################
#GENERAL UTILITY FUNCTIONS
#Get abbreviations (which eGRID uses but NEEDS does not)
#IN: states for analysis (1d list)
#OUT: map of states names to state abbreviations (dict)
def getStateAbbrevs(statesForAnalysis): 
    stateAbbreviations = {'Virginia':'VA','North Carolina':'NC','South Carolina':'SC',
                         'Georgia':'GA','Mississippi':'MS','Alabama':'AL','Louisiana':'LA',
                         'Missouri':'MO','Arkansas':'AR','Illinois':'IL',
                         'Kentucky':'KY','Tennessee':'TN','Texas':'TX'}
    statesForAnalysisAbbrev = []
    for state in statesForAnalysis:
        statesForAnalysisAbbrev.append(stateAbbreviations[state])
    return statesForAnalysisAbbrev

#Returns a list of rows to remove for values in a given column that don't 
#equal any value in valuesToKeep.
#IN: any 2d list, values in specified column to keep (1d list), col name (str)
#OUT: row indices to remove (1d list)
def identifyRowsToRemove(list2d,valuesToKeep,colName):
    headersToColsMap = mapHeadersToCols(list2d)
    colNumber = headersToColsMap[colName]
    rowsToRemove=[]
    for row in range(1,len(list2d)):
        if list2d[row][colNumber] not in valuesToKeep:
            rowsToRemove.append(row)
    return rowsToRemove

#IN: data (2d list), row idx to remove (1d list)
def removeRows(baseGenFleet,rowsToRemove):
    for row in reversed(rowsToRemove):
        baseGenFleet.pop(row)

#Returns a dictionary mapping headers to column numbers
#IN: fleet (2d list)
#OUT: map of header name to header # (dict)
def mapHeadersToCols(fleet):
    headers = fleet[0]
    headersToColsMap = dict()
    for colNum in range(len(headers)):
        header = headers[colNum]
        headersToColsMap[header] = colNum
    return headersToColsMap

#IN: data (2d list), headers to add to first row of data (1d list)
def addHeaders(fleet,listOfHeaders):
    for header in listOfHeaders:
        fleet[0].append(header)

#Returns average of values in input 1d list
def avgListVals(listOfVals):
    (total,count) = (0,0)
    for val in listOfVals:
        total += float(val)
        count += 1
    return total/count

#Removes '.0' from end of ORIS IDs in eGRID
def removeTrailingDecimalFromEgridORIS(egridORISIDs):
    for idx in range(1,len(egridORISIDs)):
        egridORISIDs[idx] = egridORISIDs[idx][:-2]

#Converts a string w/ commas in it to a float
def toNum(s):
    numSegments = s.split(',')
    result = ""
    for segment in numSegments:
        result += segment
    return float(result)

#Return row idx (or False) where list1=data1 and list2=data2
def search2Lists(list1,list2,data1,data2):
    if (data1 not in list1) or (data2 not in list2):
            return False
    for idx in range(len(list1)):
        if list1[idx] == data1 and list2[idx] == data2:
            return idx
    return False
    
#Convert specified column in 2d list to a 1-d list
def colTo1dList(data,colNum):
    listWithColData = []
    for dataRow in data:
        listWithColData.append(dataRow[colNum])
    return listWithColData

def SeperateWindSolar(generator):
    generator=generator[generator['PlantType']!='Pumped Storage']
    generator=generator[generator['PlantType']!='Energy Storage']
    filtCol=['UniqueID_Final','PlantType','Capacity (MW)','MinLoad(MWh)','OpCost($/MWh)','MinDownTime(hrs)','RampRate(MW/hr)','StartCost($)','VOM(2012$/MWh)','FOM(2012$/MW/yr)','RegOfferElig','RegOfferCost($/MW)']
    generatorForUC=generator.loc[:,filtCol]
    generatorForUC=generatorForUC.rename(columns={'UniqueID_Final':'name','Capacity (MW)':'maxcap','MinLoad(MWh)':'mincap','OpCost($/MWh)':'opcost','VOM(2012$/MWh)':'var_om','FOM(2012$/MW/yr)':'fix_om','StartCost($)':'st_cost','RampRate(MW/hr)':'ramp','MinDownTime(hrs)':'minup','RegOfferCost($/MW)':'regcost','RegOfferElig':'regelig'})
    generatorForUC.loc[generatorForUC.regelig == 0, 'regcost'] = 9999
    generatorForUC=generatorForUC.drop(columns='regelig')
    generatorDispa=generatorForUC[(generatorForUC['PlantType'] != 'Solar PV') & (generatorForUC['PlantType'] != 'Solar Thermal') & (generatorForUC['PlantType'] != 'Onshore Wind') ]
    generatorWS=generatorForUC[(generatorForUC['PlantType'] == 'Solar PV') | (generatorForUC['PlantType'] == 'Solar Thermal') |(generatorForUC['PlantType'] == 'Onshore Wind') ]
    return generatorDispa,generatorWS
################################################################################

fuelPrices = importFuelPrices('Reference case') #fuel prices
regElig = ['Steam','Combined Cycle','Storage','Geothermal','Batteries','Flywheels'] #technologies eligible to provide regulation
regCostFrac = 0.1 #regulation cost as fraction of total operating cost
stoEff,stoMinSOC = .85,0 #extra storage parameters

statesForAnalysis=['California']

powerSystemsForAnalysis=['WEC_CALN', 'WEC_LADW', 'WECC_SCE', 'WEC_SDGE', 'WECC_IID','WEC_BANC', 'WECC_PNW', 'WECC_AZ', 'WECC_NNV']
retirementYearScreen=2035
startYear=2030
compressFleet=True

# def setupGeneratorFleet(statesForAnalysis,powerSystemsForAnalysis,retirementYearScreen,
#         startYear,fuelPrices,compressFleet,regElig,regCostFrac):
#Import NEEDS (base fleet) and strip down to 1 fuel
genFleet = pd.read_excel(os.path.join('Data','needs_v6_06-30-2020.xlsx'),sheet_name='NEEDS v6_active',header=0)
genFleet.head()
    

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,Hg EMF for SUB,Hg EMF for LIG,HCL Removal,DSI Unit,DSI Online Year,CCS,CCS Removal,C2G,C2G Online Year,BART Affected Unit
0,Greene County,10_B_1,10,B,1,19.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Greene County,10_B_2,10,B,2,20.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Greene County,10_G_GT10,10,G,GT10,21.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Greene County,10_G_GT2,10,G,GT2,22.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Greene County,10_G_GT3,10,G,GT3,23.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:

genFleet['FuelType'] = genFleet['Modeled Fuels'].str.split(',', expand=True)[0]
genFleet.head()

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,Hg EMF for LIG,HCL Removal,DSI Unit,DSI Online Year,CCS,CCS Removal,C2G,C2G Online Year,BART Affected Unit,FuelType
0,Greene County,10_B_1,10,B,1,19.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas
1,Greene County,10_B_2,10,B,2,20.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas
2,Greene County,10_G_GT10,10,G,GT10,21.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas
3,Greene County,10_G_GT2,10,G,GT2,22.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas
4,Greene County,10_G_GT3,10,G,GT3,23.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas


In [3]:

#Import and extract data from EIA 860
gens,plants,storage = importEIA860()
genFleet = genFleet.merge(plants[['Plant Code','Latitude','Longitude']],left_on='ORIS Plant Code',right_on='Plant Code',how='left')
genFleet.head()



,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,DSI Online Year,CCS,CCS Removal,C2G,C2G Online Year,BART Affected Unit,FuelType,Plant Code,Latitude,Longitude
0,Greene County,10_B_1,10,B,1,19.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811
1,Greene County,10_B_2,10,B,2,20.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811
2,Greene County,10_G_GT10,10,G,GT10,21.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811
3,Greene County,10_G_GT2,10,G,GT2,22.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811
4,Greene County,10_G_GT3,10,G,GT3,23.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811


In [4]:
genFleet = genFleet.merge(storage[['Plant Code','Nameplate Energy Capacity (MWh)','Maximum Charge Rate (MW)','Maximum Discharge Rate (MW)']],left_on='ORIS Plant Code',right_on='Plant Code',how='left')
genFleet.head()

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,C2G Online Year,BART Affected Unit,FuelType,Plant Code_x,Latitude,Longitude,Plant Code_y,Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW)
0,Greene County,10_B_1,10,B,1,19.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811,NaN,NaN,NaN,NaN
1,Greene County,10_B_2,10,B,2,20.0,O/G Steam,NaN,S_SOU,Alabama,...,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811,NaN,NaN,NaN,NaN
2,Greene County,10_G_GT10,10,G,GT10,21.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811,NaN,NaN,NaN,NaN
3,Greene County,10_G_GT2,10,G,GT2,22.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811,NaN,NaN,NaN,NaN
4,Greene County,10_G_GT3,10,G,GT3,23.0,Combustion Turbine,NaN,S_SOU,Alabama,...,NaN,NaN,Natural Gas,10.0,32.6017,-87.7811,NaN,NaN,NaN,NaN


In [7]:

genFleet = genFleet.loc[genFleet['State Name'].isin(statesForAnalysis)]
genFleet = genFleet.loc[genFleet['Region Name'].str.contains('|'.join(powerSystemsForAnalysis))]

genFleet.reset_index(inplace=True,drop=True)
genFleet.head()

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,C2G Online Year,BART Affected Unit,FuelType,Plant Code_x,Latitude,Longitude,Plant Code_y,Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW)
0,Gilroy Power Plant,10034_G_GEN1,10034,G,GEN1,89552.0,Combined Cycle,NaN,WEC_CALN,California,...,NaN,NaN,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN
1,Gilroy Power Plant,10034_G_GEN2,10034,G,GEN2,NaN,Combined Cycle,NaN,WEC_CALN,California,...,NaN,NaN,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN
2,Fairhaven Power,10052_B_BLR1,10052,B,BLR1,NaN,Biomass,NaN,WEC_CALN,California,...,NaN,NaN,Biomass,10052.0,40.7995,-124.203,NaN,NaN,NaN,NaN
3,Total Energy Facilities,10091_G_G2,10091,G,G2,NaN,Non-Fossil Waste,NaN,WEC_LADW,California,...,NaN,NaN,Non-Fossil Waste,10091.0,33.7683,-118.284,NaN,NaN,NaN,NaN
4,Total Energy Facilities,10091_G_G3,10091,G,G3,NaN,Non-Fossil Waste,NaN,WEC_LADW,California,...,NaN,NaN,Non-Fossil Waste,10091.0,33.7683,-118.284,NaN,NaN,NaN,NaN


In [8]:

#Add several key parameters
genFleet = addFuelPrices(genFleet,startYear,fuelPrices)
genFleet.head()

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,BART Affected Unit,FuelType,Plant Code_x,Latitude,Longitude,Plant Code_y,Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW),FuelPrice($/MMBtu)
0,Gilroy Power Plant,10034_G_GEN1,10034,G,GEN1,89552.0,Combined Cycle,NaN,WEC_CALN,California,...,NaN,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN,3.261023
1,Gilroy Power Plant,10034_G_GEN2,10034,G,GEN2,NaN,Combined Cycle,NaN,WEC_CALN,California,...,NaN,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN,3.261023
2,Fairhaven Power,10052_B_BLR1,10052,B,BLR1,NaN,Biomass,NaN,WEC_CALN,California,...,NaN,Biomass,10052.0,40.7995,-124.203,NaN,NaN,NaN,NaN,0.000000
3,Total Energy Facilities,10091_G_G2,10091,G,G2,NaN,Non-Fossil Waste,NaN,WEC_LADW,California,...,NaN,Non-Fossil Waste,10091.0,33.7683,-118.284,NaN,NaN,NaN,NaN,0.000000
4,Total Energy Facilities,10091_G_G3,10091,G,G3,NaN,Non-Fossil Waste,NaN,WEC_LADW,California,...,NaN,Non-Fossil Waste,10091.0,33.7683,-118.284,NaN,NaN,NaN,NaN,0.000000


In [9]:

genFleet = addEmissionsRates(genFleet)
genFleet.head() 

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,FuelType,Plant Code_x,Latitude,Longitude,Plant Code_y,Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW),FuelPrice($/MMBtu),CO2EmRate(lb/MMBtu)
0,Gilroy Power Plant,10034_G_GEN1,10034,G,GEN1,89552.0,Combined Cycle,NaN,WEC_CALN,California,...,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN,3.261023,117.0
1,Gilroy Power Plant,10034_G_GEN2,10034,G,GEN2,NaN,Combined Cycle,NaN,WEC_CALN,California,...,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN,3.261023,117.0
2,Fairhaven Power,10052_B_BLR1,10052,B,BLR1,NaN,Biomass,NaN,WEC_CALN,California,...,Biomass,10052.0,40.7995,-124.203,NaN,NaN,NaN,NaN,0.000000,91.9
3,Total Energy Facilities,10091_G_G2,10091,G,G2,NaN,Non-Fossil Waste,NaN,WEC_LADW,California,...,Non-Fossil Waste,10091.0,33.7683,-118.284,NaN,NaN,NaN,NaN,0.000000,160.1
4,Total Energy Facilities,10091_G_G3,10091,G,G3,NaN,Non-Fossil Waste,NaN,WEC_LADW,California,...,Non-Fossil Waste,10091.0,33.7683,-118.284,NaN,NaN,NaN,NaN,0.000000,160.1


In [10]:

if compressFleet == True: genFleet = performFleetCompression(genFleet)
genFleet.head()


,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,FuelType,Plant Code_x,Latitude,Longitude,Plant Code_y,Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW),FuelPrice($/MMBtu),CO2EmRate(lb/MMBtu)
0,Gilroy Power Plant,10034_G_GEN1,10034,G,GEN1,89552.0,Combined Cycle,NaN,WEC_CALN,California,...,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN,3.261023,117.0
1,Gilroy Power Plant,10034_G_GEN2,10034,G,GEN2,NaN,Combined Cycle,NaN,WEC_CALN,California,...,Natural Gas,10034.0,37.0001,-121.537,NaN,NaN,NaN,NaN,3.261023,117.0
2,Gosselin Hydro Plant,10128_G_GEN1,10128,G,GEN1,NaN,Hydro,NaN,WEC_CALN,California,...,Hydro,10128.0,40.37,-123.435,NaN,NaN,NaN,NaN,0.000000,0.0
3,Gosselin Hydro Plant,10128_G_GEN2,10128,G,GEN2,NaN,Hydro,NaN,WEC_CALN,California,...,Hydro,10128.0,40.37,-123.435,NaN,NaN,NaN,NaN,0.000000,0.0
4,Isabella Hydro Project,10139_G_GEN1,10139,G,GEN1,NaN,Hydro,NaN,WECC_SCE,California,...,Hydro,10139.0,35.6439,-118.482,NaN,NaN,NaN,NaN,0.000000,0.0


In [11]:
genFleet = addUnitCommitmentParameters(genFleet,'PhorumUCParameters.csv') 
genFleet.head()

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,Plant Code_y,Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW),FuelPrice($/MMBtu),CO2EmRate(lb/MMBtu),MinDownTime(hrs),RampRate(MW/hr),StartCost($),MinLoad(MWh)
0,Gilroy Power Plant,10034_G_GEN1,10034,G,GEN1,89552.0,Combined Cycle,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,3.261023,117.0,4.0,54.315,8677.634504,34.0
1,Gilroy Power Plant,10034_G_GEN2,10034,G,GEN2,NaN,Combined Cycle,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,3.261023,117.0,4.0,19.170,3062.694531,12.0
2,Gosselin Hydro Plant,10128_G_GEN1,10128,G,GEN1,NaN,Hydro,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,0.200,0.000000,0.0
3,Gosselin Hydro Plant,10128_G_GEN2,10128,G,GEN2,NaN,Hydro,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,0.200,0.000000,0.0
4,Isabella Hydro Project,10139_G_GEN1,10139,G,GEN1,NaN,Hydro,NaN,WECC_SCE,California,...,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,5.900,0.000000,0.0


In [12]:

genFleet = addUnitCommitmentParameters(genFleet,'StorageUCParameters.csv')
genFleet.head()

,Plant Name,UniqueID_Final,ORIS Plant Code,Boiler/Generator/Committed Unit,Unit ID,CAMD Database UnitID,PlantType,Combustion Turbine/IC Engine,Region Name,State Name,...,Plant Code_y,Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW),FuelPrice($/MMBtu),CO2EmRate(lb/MMBtu),MinDownTime(hrs),RampRate(MW/hr),StartCost($),MinLoad(MWh)
0,Gilroy Power Plant,10034_G_GEN1,10034,G,GEN1,89552.0,Combined Cycle,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,3.261023,117.0,0.0,85.0,0.0,85.0
1,Gilroy Power Plant,10034_G_GEN2,10034,G,GEN2,NaN,Combined Cycle,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,3.261023,117.0,0.0,30.0,0.0,30.0
2,Gosselin Hydro Plant,10128_G_GEN1,10128,G,GEN1,NaN,Hydro,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,0.2,0.0,0.2
3,Gosselin Hydro Plant,10128_G_GEN2,10128,G,GEN2,NaN,Hydro,NaN,WEC_CALN,California,...,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,0.2,0.0,0.2
4,Isabella Hydro Project,10139_G_GEN1,10139,G,GEN1,NaN,Hydro,NaN,WECC_SCE,California,...,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,5.9,0.0,5.9


In [ ]:

genFleet = addRandomOpCostAdder(genFleet)
genFleet = addVOMandFOM(genFleet) 
genFleet = calcOpCost(genFleet)
genFleet = addRegResCostAndElig(genFleet,regElig,regCostFrac)
#Add retirement rows
for c in ['YearAddedCE','Retired','YearRetiredByCE','YearRetiredByAge']: genFleet[c]=False
    # return genFleet 

# genFleet = setupGeneratorFleet(['California'],['WEC_CALN', 'WEC_LADW', 'WECC_SCE', 'WEC_SDGE', 'WECC_IID','WEC_BANC', 'WECC_PNW', 
#             'WECC_AZ', 'WECC_NNV'],2035,2030,fuelPrices,True, regElig, regCostFrac) #,stoEff,stoMinSOC
# genFleet.to_csv('genFleet.csv')
generatorDispa,generatorWS=SeperateWindSolar(genFleet)
# generatorDispa.drop(['PlantType'],axis=1)
generatorDispa.to_csv('GeneratorRaw2019.csv',index=False)
# generatorWS.to_csv('WindSolarConstraintRaw.csv',index=False)

In [5]:
df1.head()

,INTERVALSTARTTIME_GMT,RENEWABLE_TYPE,MW
0,2019-04-18T08:00:00-00:00,Solar,-2.91828
1,2019-04-18T09:00:00-00:00,Solar,-2.98123
2,2019-04-18T10:00:00-00:00,Solar,-2.96208
3,2019-04-18T11:00:00-00:00,Solar,-2.95255
4,2019-04-18T12:00:00-00:00,Solar,-2.93420


In [6]:
Solar=df1
Solar['MW']=df1['MW']+df3['MW']+df5['MW']
Solar=Solar.drop(['INTERVALSTARTTIME_GMT','RENEWABLE_TYPE'],axis=1)

ValueError: cannot reindex from a duplicate axis

In [8]:

from datetimerange import DateTimeRange 
def GetSolarWindRawYearlyData():
    directory = r'/Users/meiyewang/Desktop/code/data/windsolar/ACTUAL/2019'
    df1=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df2=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df3=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df4=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df5=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    for filename in os.listdir(directory):
        if filename.endswith(".csv") :
            df=pd.read_csv(os.path.join(directory, filename))
            df=pd.read_csv(os.path.join(directory, filename))
            df1=pd.concat([df1,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df2=pd.concat([df2,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df3=pd.concat([df3,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df4=pd.concat([df4,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df5=pd.concat([df5,df[(df['TRADING_HUB']=='ZP26') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        else:
            continue
    return df1,df2,df3,df4,df5

def SortTimeOrder(df):
    timelist=[datetime.datetime.strptime(df.iloc[i,0][:16], "%Y-%m-%dT%H:%M") for i in range(df.shape[0])]
    df["INTERVALSTARTTIME_GMT"]=timelist
    df_sort=df.sort_values(by="INTERVALSTARTTIME_GMT")
    df_sort.index=df_sort["INTERVALSTARTTIME_GMT"]
    return df_sort


ValueError: Invalid fill method. Expecting pad (ffill), backfill (bfill) or nearest. Got 99999

In [25]:

def FillMissingData(df_sort):
    
    time_range = DateTimeRange("2019-01-01 08:00:00", "2020-01-01 07:00:00")
    completeIndex= time_range.range(datetime.timedelta(hours=1))
    
    df_sort_complete=df_sort.reindex(completeIndex)
    df_sort_complete=df_sort_complete.interpolate(method='linear')
    return df_sort_complete


df1,df2,df3,df4,df5=GetSolarWindRawYearlyData()

df1_complete=FillMissingData(SortTimeOrder(df1))
df2_complete=FillMissingData(SortTimeOrder(df2))
df3_complete=FillMissingData(SortTimeOrder(df3))
df4_complete=FillMissingData(SortTimeOrder(df4))
df5_complete=FillMissingData(SortTimeOrder(df5))


In [27]:

def CompileWindSolar(df1_complete,df2_complete,df3_complete,df4_complete,df5_complete):
    Solar=df1_complete
    Solar['MW']=df1_complete['MW']+df3_complete['MW']+df3_complete['MW']
    Solar.index=range(8760)
    Solar=Solar.drop(['INTERVALSTARTTIME_GMT','RENEWABLE_TYPE'],axis=1)

    Wind=df2_complete
    Wind['MW']=df2_complete['MW']+df4_complete['MW']
    Wind.index=range(8760)
    Wind=Wind.drop(['INTERVALSTARTTIME_GMT','RENEWABLE_TYPE'],axis=1)
    return Solar,Wind



Solar,Wind=CompileWindSolar(df1_complete,df2_complete,df3_complete,df4_complete,df5_complete)



In [29]:
Solar[pd.isna(Solar['MW'])]

,MW


In [1]:
import pandas as pd 
solarwind=pd.read_csv('WindSolarConstraintRaw.csv').groupby('PlantType')

In [2]:
solarwind.sum()

,maxcap,mincap,opcost,minup,ramp,st_cost,var_om,fix_om,regcost
PlantType,,,,,,,,,
Onshore Wind,5763.8,0.0,2.862469,0.0,5763.8,0.0,0.0,4548250.0,1149885.0
Solar PV,9362.8,0.0,16.560828,0.0,9362.8,0.0,0.0,18842250.0,6789321.0
Solar Thermal,1281.0,0.0,0.000000,0.0,1281.0,0.0,0.0,0.0,129987.0


In [3]:
solarwind.mean()

,maxcap,mincap,opcost,minup,ramp,st_cost,var_om,fix_om,regcost
PlantType,,,,,,,,,
Onshore Wind,50.120000,0.0,0.024891,0.0,50.120000,0.0,0.0,39550.0,9999.0
Solar PV,13.789102,0.0,0.024390,0.0,13.789102,0.0,0.0,27750.0,9999.0
Solar Thermal,98.538462,0.0,NaN,0.0,98.538462,0.0,NaN,NaN,9999.0


In [4]:
solarwind.count()

,name,maxcap,mincap,opcost,minup,ramp,st_cost,var_om,fix_om,regcost
PlantType,,,,,,,,,,
Onshore Wind,115,115,115,115,115,115,115,115,115,115
Solar PV,679,679,679,679,679,679,679,679,679,679
Solar Thermal,13,13,13,0,13,13,13,0,0,13
